### Prerequisites
#### 1) cobrapy
One can install cobrapy (https://cobrapy.readthedocs.io/en/latest/)

through Anaconda: conda install -c bioconda cobra
OR
through pip: pip install cobra

More infor available here: https://github.com/opencobra/cobrapy

#### 2) iML1515.json
It is the latest E. coli Genome scale metabolic model. This can be obtained through Bigg database http://bigg.ucsd.edu/

#### 3) Gurobi solver
https://www.gurobi.com/downloads/gurobi-software/

In [1]:
from AuxoFind import *
import os

In [2]:
directory = os.getcwd()
model = cobra.io.load_json_model('%s/iML1515.json'%directory) # replace with iML1515 directory

Academic license - for non-commercial use only


**You can get the conditionally essential genes (CEGs) for a model using**

In [9]:
M9_medium = {'EX_h2o_e', 'EX_mn2_e', 'EX_glc__D_e', 'EX_ni2_e', 'EX_sel_e', 'EX_o2_e', 'EX_cu2_e', 'EX_pi_e', 'EX_h_e', 'EX_mobd_e', 
             'EX_tungs_e', 'EX_na1_e', 'EX_zn2_e', 'EX_ca2_e', 'EX_cl_e', 'EX_fe3_e', 'EX_so4_e', 'EX_slnt_e', 'EX_mg2_e', 'EX_co2_e',
             'EX_k_e', 'EX_nh4_e', 'EX_fe2_e', 'EX_cobalt2_e'}
CEGs = get_CEGs(model, growth_medium = M9_medium)
print('The model predicts %d conditionally essential genes in M9+glucose minimal medium.'%(len(CEGs)))

The model predicts 109 conditionally essential genes in M9+glucose minimal medium.


**Assuming you have a list of missing/disrupted genes which you can obtain via any method of your choice**

In [4]:
missing_genes = {'b2677', 'b0243', 'b1262'}
auxotrophies = get_auxotrophies(model, missing_genes, method = 'single_search')

auxotrophies

Number of solutions: 50


/home/yara/Documents/Auxotrophy/code_for_publication/AuxoFind.py:118 UserWarning: The gene (b2677) is not conditionally essential. Its knock-out has no effect on growth.


Number of solutions: 50


{'b0243': [['EX_progly_e'], ['EX_pro__L_e']],
 'b1262': [['EX_indole_e'], ['EX_trp__L_e']]}

**Notice the warning, one of the genes (b2677) is not a CEG. If you remove the gene from the list of missing CEGs, the warning disappears**

In [5]:
missing_genes = {'b0243', 'b1262'}
auxotrophies = get_auxotrophies(model, missing_genes, method = 'single_search')

auxotrophies

Number of solutions: 50
Number of solutions: 50


{'b0243': [['EX_progly_e']], 'b1262': [['EX_indole_e'], ['EX_trp__L_e']]}

**The function returns a dictionary mapping each gene to the nutrients that the model needs to simulate growth.
For the single_search option, a list of all solutions is provided. When there are two nutrients that are simulatenous required, they are featured within the same list, 
When two solutions are equivalent, they are featured in different lists. Here's another way to present the results**

In [6]:
auxotrophies_t = {model.genes.get_by_id(gene).name: ' OR '.join([' AND '.join([model.metabolites.get_by_id(rx.replace('EX_','')).name for rx in rx_l]) for rx_l in rxs]) for gene, rxs in auxotrophies.items()}
auxotrophies_t

{'proA': 'L-Prolinylglycine', 'trpC': 'Indole OR L-Tryptophan'}

**What if you're only interested in getting one single solution? In that case just remove the method setting and it will return to the default (optimal_solution)**

In [7]:
auxotrophies = get_auxotrophies(model, missing_genes)

auxotrophies

{'b0243': dict_keys(['EX_progly_e']), 'b1262': dict_keys(['EX_trp__L_e'])}

**You can also add a list of nutrients that you would like to exclude from the solution. For example L-prolinylglycine might not provide sufficient resolution, and you might prefer to know whether the strain is auxotrophic for L-proline specifically or L-glycine.**

In [8]:
auxotrophies = get_auxotrophies(model, missing_genes, method = 'single_search', force_to_zero = ['EX_progly_e'])
auxotrophies_t = {model.genes.get_by_id(gene).name: ' OR '.join([' AND '.join([model.metabolites.get_by_id(rx.replace('EX_','')).name for rx in rx_l]) for rx_l in rxs]) for gene, rxs in auxotrophies.items()}
auxotrophies_t

Number of solutions: 50
Number of solutions: 50


{'proA': 'L-Proline', 'trpC': 'Indole OR L-Tryptophan'}

**type ?function_name to get specifics about function settings**

In [18]:
?get_auxotrophies